In [1]:
import tweepy # access the twitter API
import time # manipulate time values
import pandas as pd # data analysis and manipulation
from tqdm import tqdm, notebook # allow to output a smart progress bars or meters

In [2]:
consumer_key = "7SZ0A7V2Dd77NWeLSshqoauNM"
consumer_secret = "DLKteB78caT6FssgIgeUQsCbaHzFUpquZa8iMfWqH2EXxRUZ9B"
callback_uri = "oob"

In [3]:
# get auth anf click on the link
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=-ljL9AAAAAABXjIGAAABfr7tkT0


In [4]:
api = tweepy.API(auth, wait_on_rate_limit = True)

In [5]:
# paste the pin value by answering the question
user_pin_input = input("What is the pin value? ")

What is the pin value? 6198188


In [8]:
user_pin_input

'6198188'

In [9]:
# get authorizations
auth.get_access_token(user_pin_input)

('1371248311324323842-7aQb6lutPq86JrjCsINIrnPip3idin',
 'yDAnyJ7T7xfxPIux3vLj4BswNiQK6gZ4xOGXoVi0hyxcK')

In [10]:
# collect tweets with the hashtag #covid
search_words = "#covid -filter:retweets"
date_since = "2022-01-31"
date_until = "2022-02-01"

tweets = tweepy.Cursor(api.search, q = search_words, lang = "en", since = date_since, until = date_until).items()

In [11]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)


5509it [30:18,  3.03it/s]


In [12]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 5509


In [13]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|████████████████████████████████████████████████████████████████████████████| 5509/5509 [1:30:51<00:00,  1.01it/s]


In [14]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Ϯஐ รครรคfгครร Ϯஐ,But it is our duty to live.,I have brain cells lovelier than my eyes. I ♥️...,2009-02-21 16:40:08,2334,1670,63453,False,2022-01-31 23:59:59,"Hey, @washingtonpost \n\n👀 Just look at all th...",[covid],Twitter for iPhone,False
0,APCA,"Phoenix, AZ",Official Twitter account of APCA - a community...,2010-01-22 18:57:07,549,678,645,False,2022-01-31 23:59:11,#COVID boosters are available for everyone 12+...,[COVID],SocialPilot.co,False
0,Nathan Joyner,"Los Angeles, CA",Global Venture Captial and Private Equity/Busi...,2015-05-18 20:52:29,48,11,798,False,2022-01-31 23:59:04,7 Day Covid Deaths per 1K Population by County...,None,smcapplication,False
0,Nathan Joyner,"Los Angeles, CA",Global Venture Captial and Private Equity/Busi...,2015-05-18 20:52:29,48,11,798,False,2022-01-31 23:57:32,7 Day Covid Deaths per 1K Population by County...,None,smcapplication,False
0,World Alert,"Karlsruhe, Germany","Alerting the world, with the current happening...",2018-03-20 14:08:51,324,34,9,False,2022-01-31 23:57:12,There are currently 51candidates\nOut of which...,"[coronavirus, COVID, Covid, vaccine]",corona-tracker-app,False


In [15]:
tweets_df.to_csv("covid_tweets_31.csv", index = False)